In [0]:
#r "nuget:Microsoft.ML , 1.1.0"
#r "nuget:Microsoft.ML.CpuMath , 1.1.0"

In [1]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.Transforms;
using static Microsoft.ML.DataOperationsCatalog;

In [2]:
public class SentimentIssue
{
  [LoadColumn(0)]
  public bool Label { get; set; }
  [LoadColumn(2)]
  public string Text { get; set; }
}

In [3]:
public class SentimentPrediction
{
   [ColumnName("PredictedLabel")]
   public bool Prediction { get; set; }
   
   public float Probability { get; set; }

   public float Score { get; set; }
}

In [4]:
var mlContext = new MLContext();

In [5]:
IDataView dataView = mlContext.Data.LoadFromTextFile<SentimentIssue>(@"/Users/lujianhui/Desktop/wikiDetoxAnnotated40kRows.tsv", hasHeader: true);

TrainTestData trainTestSplit = mlContext.Data.TrainTestSplit(dataView, testFraction: 0.2);
IDataView trainingData = trainTestSplit.TrainSet;
IDataView testData = trainTestSplit.TestSet;

In [6]:
var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText(outputColumnName: "Features", inputColumnName: nameof(SentimentIssue.Text));

In [7]:
var trainer = mlContext.BinaryClassification.Trainers.SdcaLogisticRegression(labelColumnName: "Label", featureColumnName: "Features");
var trainingPipeline = dataProcessPipeline.Append(trainer);

In [ ]:
ITransformer trainedModel = trainingPipeline.Fit(trainingData);